<a href="https://colab.research.google.com/github/PATELOM925/Automatic-Paper-Checker/blob/main/Complete_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install pytesseract opencv-python pillow spacy
!python -m spacy download en_core_web_sm


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
3 not fully installed or removed.
After this operation, 0 B of additional disk space will be used.
Setting up tesseract-ocr-eng (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-osd (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr (4.1.1-2.1build1) ...
Processing triggers for man-db (2.10.2-1) ...
2023-10-13 03:49:14.364049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 03:49:15.571721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files
import cv2
import pytesseract
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np

# Use the files.upload() method to upload the student's and teacher's images
uploaded_student = files.upload()
uploaded_teacher = files.upload()

# Load spaCy model (you can choose a different language model if needed)
nlp = spacy.load("en_core_web_sm")

# Initialize NLTK sentiment analyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Preprocess image
def preprocess_image(image_path):
    # Reading image
    image = cv2.imread(image_path, 0)  # Read as grayscale

    # Adaptive thresholding
    binary_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours
    min_contour_area = 100
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    # Drawing contours on the original image
    segmented_image = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), filtered_contours, -1, (0, 255, 0), 2)

    return segmented_image

# Function to perform OCR on the input image
def perform_ocr(image_path):
    segmented_image = preprocess_image(image_path)
    recognized_text = pytesseract.image_to_string(segmented_image, config='--psm 6')
    return recognized_text

# Function to process the recognized text using NLP techniques
def process_text(recognized_text):
    # Tokenization using spaCy
    doc = nlp(recognized_text)

    # Sentiment analysis using NLTK
    sentiment_scores = sia.polarity_scores(recognized_text)
    sentiment = "Positive" if sentiment_scores["compound"] > 0 else "Negative" if sentiment_scores["compound"] < 0 else "Neutral"

    return recognized_text, sentiment

# Student's image path
student_image_path = list(uploaded_student.keys())[0]

# Teacher's image path
teacher_image_path = list(uploaded_teacher.keys())[0]

# Calling OCR function for the student and teacher images
student_recognized_text = perform_ocr(student_image_path)
teacher_recognized_text = perform_ocr(teacher_image_path)

# Processing the recognized text for the student and teacher
student_text, student_sentiment = process_text(student_recognized_text)
teacher_text, teacher_sentiment = process_text(teacher_recognized_text)

# Calculate semantic similarity between student and teacher text
student_doc = nlp(student_text)
teacher_doc = nlp(teacher_text)
similarity = student_doc.similarity(teacher_doc)

# Print results
print("Student Recognized Text:", student_text)
print("Student Sentiment:", student_sentiment)
print("\nTeacher Recognized Text:", teacher_text)
print("Teacher Sentiment:", teacher_sentiment)
print("\nSemantic Similarity Score:", similarity)


Saving harsh.png to harsh.png


Saving harsh.png to harsh (1).png


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Student Recognized Text: Deliciovs eghsti tot for: tone in you! Favorite
)eNe Dted bor octetiM’ nors soevvle? and.
{ Cote BS" Great OF ot sale ort

Student Sentiment: Positive

Teacher Recognized Text: Deliciovs eghsti tot for: tone in you! Favorite
)eNe Dted bor octetiM’ nors soevvle? and.
{ Cote BS" Great OF ot sale ort

Teacher Sentiment: Positive

Semantic Similarity Score: 1.0
